## Problem Statement

The Government of Tanzania and water-focused NGOs face limited maintenance budgets while managing thousands of water wells across the country. Some wells are fully functional, some require repair, and others have completely failed. The challenge is to identify which wells are at risk of failure so that maintenance and investment resources can be allocated efficiently and proactively.

This project builds a machine learning classification model to predict the operational status of a water well—functional, in need of repair, or non-functional—based on physical characteristics, installation details, and environmental factors.


### Context: Population and Water Access in Tanzania

Tanzania has a population of over 61 million people (2022 Census), the majority of whom rely on public and community-managed water points for daily water access. Even small-scale water well failures can affect thousands of people, particularly in rural and underserved areas.

<img src="Images/Tanzania-Census-results-2022-2.png" width="800">


## Data Preparation
**IMPORT LIABRARIES AND DATA SET**



In [ ]:
import pandas as pd

# Load both parts of the training data
features = pd.read_csv("Training_set_labels.csv")      # the big table y
labels   = pd.read_csv("Test_set_values.csv")      # id + status_group

# Merge on 'id' 
df_train = pd.merge(
    features,
    labels,
    on='id',
    how='inner'          # almost always 'inner' here — should keep all 59400 rows
)

# Quick checks after merge
print("Shape after merge:", df_train.shape) # should be (59400, 41)


Shape after merge: (59400, 41)


In [ ]:
# looking at status_group dynamics
df_train['status_group'].value_counts()

status_group
functional                 32259
non functional             22824
functional needs repair     4317
Name: count, dtype: int64

In [ ]:
# getting info about the data set
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55763 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59398 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

- The training dataset contains **59,400 rows** and **41 columns**.
- Column types:
  - **Numeric (int64 / float64):** 10 columns
  - **Categorical (object):** 31 columns
- Key observations:
  - The **target variable** is `status_group` (categorical) with 3 classes: `functional`, `functional needs repair`, `non functional`.
  - Several categorical columns have missing values:
    - `scheme_name` (~49% missing) → may require special handling or dropping.
    - `funder`, `installer`, `public_meeting`, `permit` (~5–6% missing) → can be imputed with mode or 'Unknown'.
 

## DATA CLEANING 
Data cleaning for this dataset focused on ensuring completeness, consistency, and readiness for modeling. The key steps include:

1. **Handling Missing Values**
   - Columns with small missing percentages (e.g., `funder`, `installer`, `public_meeting`, `permit`) were filled with `'Unknown'`.
   - Columns with high missing percentages (e.g., `scheme_name` ~49%) were either carefully imputed using mode/label encoding or dropped if not informative.
   - Numeric columns were checked for missing or zero values and imputed with median or left as is if zero was meaningful (e.g., `amount_tsh=0`).

2. **Encoding Categorical Variables**
   - High-cardinality categorical columns (`funder`, `installer`, `scheme_name`) were encoded using **Label Encoding** to avoid creating thousands of dummy columns.
   - Low-cardinality categorical columns (`basin`, `region`, `management_group`, `payment_type`) were encoded using **One-Hot Encoding** to preserve interpretability.

3. **Feature Selection**
   - Non-informative ID/text columns (`id`, `wpt_name`, `subvillage`) were dropped to reduce noise.
   - Features relevant to waterpoint functionality (`construction_year`, `gps_height`, `population`, `extraction_type`, `management`, `water_quality`) were retained.

4. **Target Preparation**
   - The target variable `status_group` was label-encoded to numeric values for model training:
     - `functional` → 0
     - `functional needs repair` → 1
     - `non functional` → 2

5. **Sanity Checks**
   - Verified dataset shapes and column types after cleaning.
   - Ensured no remaining missing values in features used for modeling.

**Outcome:** A clean, numeric-ready dataset suitable for training classification models like Random Forest, XGBoost, or Logistic Regression.
